# Três Perceptron

### Dados e Tratamento

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

N = 1000
seed = 8
np.random.seed(seed)

x1 = np.random.randint(0,2,N, dtype=bool).reshape(N,1)
x2 = np.random.randint(0,2,N, dtype=bool).reshape(N,1)

# Função objetivo (Final desejado, XOR).
y = x1 ^ x2

# Labels do primeiro perceptron.
y1 = x1 & (~x2)

# Labels do segundo perceptron.
y2 = (~x1) & x2

# Atributos e labels do terceito perceptron.
X3 = np.c_[y1, y2]
y3 = y1 | y2

x1 = x1 + 0.1*np.random.randn(N,1)
x2 = x2 + 0.1*np.random.randn(N,1)
X = np.c_[x1, x2]

idx0 = np.argwhere(y.ravel() == 0)
idx1 = np.argwhere(y.ravel() == 1)

plt.plot(x1[idx0.ravel()], x2[idx0.ravel()], '.', label='Class 0')
plt.plot(x1[idx1.ravel()], x2[idx1.ravel()], 'rx', label='Class 1')
plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.title('Noisy XOR function')
plt.legend()
plt.show()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.3, random_state=seed)
X1_train = np.c_[np.ones((len(y1_train), 1)), X1_train]
X1_test = np.c_[np.ones((len(y1_test), 1)), X1_test]

## Percpetrons

### Perceptron 1

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

per1 = Perceptron(fit_intercept=False, random_state=seed)
per1.fit(X1_train, y1_train.ravel())
y_pred1 = per1.predict(X1_test)
acc = accuracy_score(y1_test, y_pred1)
print('Acurácia perceptron #1:',acc)

X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.3, random_state=seed)
X2_train = np.c_[np.ones((len(y2_train), 1)), X2_train]
X2_test = np.c_[np.ones((len(y2_test), 1)), X2_test]

### Perceptron 2

In [ ]:
per2 = Perceptron(fit_intercept=False, random_state=seed)
per2.fit(X2_train, y2_train.ravel())
y_pred2 = per2.predict(X2_test)
acc = accuracy_score(y2_test, y_pred2)
print('Acurácia perceptron #2:',acc)

X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3, random_state=seed)
X3_train = np.c_[np.ones((len(y3_train), 1)), X3_train]
X3_test = np.c_[np.ones((len(y3_test), 1)), X3_test]

### Perceptron 3

In [ ]:
per3 = Perceptron(fit_intercept=False, random_state=seed)
per3.fit(X3_train, y3_train.ravel())
y_pred3 = per3.predict(X3_test)
acc = accuracy_score(y3_test, y_pred3)
print('Acurácia perceptron #3:',acc)

### Definição de uma classe para Classificador

In [ ]:
class Classifier():

    def __init__(self, per1, per2, per3):
        self.per1 = per1
        self.per2 = per2
        self.per3 = per3

    def predict(self, X):
        N = X.shape[0]  # get the number of lines.
        y1 = self.per1.predict(X).reshape(N, 1)
        y2 = self.per2.predict(X).reshape(N, 1)
        X3 = np.c_[np.ones((len(y1), 1)), y1, y2]
        y3 = self.per3.predict(X3)
        return y3

### Instanciando o Classificador

In [ ]:
clf = Classifier(per1, per2, per3)
X_ = np.c_[np.ones((N,1)), X]
y_pred = clf.predict(X_)

acc = accuracy_score(y, y_pred)
print('Acurácia da combinação dos 3 perceptrons:',acc)

### Saídas

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .01
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

attribute_matrix = np.c_[np.ones((len(xx.ravel()), 1)), xx.ravel(), yy.ravel()]
Z = clf.predict(attribute_matrix)
Z = Z.reshape(xx.shape)
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Greens, shading='auto')

plt.plot(X[idx0,0], X[idx0,1], '.', markersize=8, label='Class 0')
plt.plot(X[idx1,0], X[idx1,1], 'rx', markersize=8, label='Class 1')
plt.xlabel('$x_1$', fontsize=14)
plt.ylabel('$x_2$', fontsize=14)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.show()

X_test = np.c_[np.ones((len(y_test), 1)), X_test]
y_pred = clf.predict(X_test)

mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=range(2), yticklabels=range(2), cmap="Blues")
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()